### Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
# Importing LabelEncoder from Sklearn
# library from preprocessing Module.

import librosa

### Loading the dataset using Pandas
The data is found in development.csv (development set): a comma-separated values file containing the records from the development set. This portion does have the action and object columns, which you should use to obtain the labels to train and validate your models.
The dataset consists in a collection of audio file in a WAV format. 

Each record is characterized by several attributes. The following is a short description for each of them.
- path: the path of the audio file.
- speakerId: the id of the speaker.
- action: the type of action required through the intent.
- object: the device involved by intent.
- Self-reported fluency level: the speaking fluency of the speaker.
- First Language spoken: the first language spoken by the speaker.
- Current language used for work/school: the main language spoken by the speaker during daily activities.
- gender: the gender of the speaker.
- ageRange: the age range of the speaker.

In [ ]:
df = pd.read_csv("dsl_data/development.csv")
df2 = pd.read_csv("out.csv")

In [ ]:
y = df.values[:,3:5].sum(axis=1)
y

In [ ]:
from algorithms import encode_columns

column_names = ["Self-reported fluency level ","First Language spoken", "Current language used for work/school", "speakerId", "gender","ageRange"]
encode_columns(df,column_names)

In [ ]:
df["mfccs"] = df.apply(lambda row: librosa.feature.mfcc(y=librosa.load(row["path"])[0], sr=librosa.load(row["path"])[1]), axis=1)

In [ ]:
df["mfccs_mean"] = df["mfccs"].apply(lambda x: np.mean(x))

In [ ]:
X = df2[["speakerId","Self-reported fluency level ","First Language spoken","Current language used for work/school","gender","ageRange", "mfccs_mean"]]
X

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# create an instance of the SVM model
clf = SVC()

# train the model on the training data
clf.fit(X_train, y_train)
# # predict the target values for the test data
y_pred = clf.predict(X_test)

# # evaluate the model using metrics such as accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, random_state=0)

# fit the classifier to the training data
clf.fit(X_train, y_train)

# make predictions on the test set
y_pred = clf.predict(X_test)

# calculate the accuracy of the model
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy

In [ ]:
## Implementation using k-fold

from sklearn.model_selection import KFold, cross_val_score
# define the number of folds and whether to shuffle the data
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# create an instance of the SVM model
clf = SVC()

# use cross_val_score function to perform k-fold cross-validation
scores = cross_val_score(clf, X, y, cv=kf, scoring='accuracy')

# print the mean accuracy and standard deviation
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std()))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle

clf = RandomForestClassifier(n_estimators=100, random_state=0)

# shuffle the data before performing k-fold cross validation
X, y = shuffle(X, y)

# perform k-fold cross validation with 5 folds
scores = cross_val_score(clf, X, y, cv=5)

# calculate the mean accuracy of the model across all folds
accuracy = np.mean(scores)
print("Accuracy:", accuracy)
